In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/aa406165/Git/qiskit-core/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [5]:
from qiskit.providers.aer import AerSimulator
backend = AerSimulator()

In [6]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=(1.7786666666666666+0j), variance=(11.140155555555555+0j), confidence_interval=[((1.685911814800926+0j), (1.8714215185324072+0j))])

In [7]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=(1.859375+0j), variance=(11.079803466796875+0j), confidence_interval=[((1.7013771658005008+0j), (2.017372834199499+0j))])

In [8]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for this backend: aer_simulator


none


CompositeResult(items=[ExpectationValueResult(value=(1.8119999999999998+0j), variance=(11.076720000000002+0j), confidence_interval=[((1.7196992349721762+0j), (1.9043007650278234+0j))]), ExpectationValueResult(value=0.0, variance=0.0, confidence_interval=[(0.0, 0.0)])])

### Exact simulation by SaveExpectationValueVariance

In [9]:
from qiskit.evaluators import ExactExpectationValue

In [10]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 9.42 ms, sys: 704 µs, total: 10.1 ms
Wall time: 10.1 ms


ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [11]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 3.81 ms, sys: 996 µs, total: 4.81 ms
Wall time: 4.54 ms


ExpectationValueResult(value=0.15611896641425527, variance=5.356814314603938, confidence_interval=None)

### Transpiled Circuits

In [12]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
# expval.set_transpile_options(initial_layout=[1, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
               ┌────┐┌──────────────┐┌────┐┌────────┐     ┌────┐»
      q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤┌─┴─┐├────┤»
      q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘└───┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 3 ─────────────────────────────────────────────────»
                                                                »
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c9: 2/═════════════════════════════════════════════════»
                                                                »
«               ┌──────────────┐┌────┐┌────────┐     ┌────┐┌──────────────┐»
«      q_0 -> 0 ┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─

### Transpile options and Run options

In [13]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=(1.5210701190765445+0j), variance=(12.011678548031021+0j), confidence_interval=[((1.5156457698115229+0j), (1.5264944683415662+0j))])

In [14]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=(1.516357421875+0j), variance=(12.050503671169281+0j), confidence_interval=[((1.4562694077614355+0j), (1.5764454359885645+0j))])

In [15]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=(1.64+0j), variance=(12.009511111111111+0j), confidence_interval=[((1.326503196664666+0j), (1.9534968033353337+0j))])

### Composite Evaluator

In [16]:
from qiskit.evaluators import JointEvaluator

In [17]:
len(expval.transpiled_circuits)

3

In [18]:
joint_evaluator = JointEvaluator([expval, expval, expval])

In [19]:
len(joint_evaluator.transpiled_circuits) # 3 × 3

9

In [20]:
joint_evaluator.evaluate([0, 1, 1, 2, 3, 5])

CompositeResult(items=[ExpectationValueResult(value=(1.556640625+0j), variance=(11.943098902702332+0j), confidence_interval=[((1.4969504728178185+0j), (1.6163307771821815+0j))]), ExpectationValueResult(value=(1.542724609375+0j), variance=(11.941121995449066+0j), confidence_interval=[((1.483053039379968+0j), (1.602396179370032+0j))]), ExpectationValueResult(value=(1.54443359375+0j), variance=(11.978637337684631+0j), confidence_interval=[((1.4846131380151335+0j), (1.6042540494848665+0j))])])

In [21]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=(1.507080078125+0j), variance=(12.055359065532684+0j), confidence_interval=[((1.4469819739206327+0j), (1.5671781823293673+0j))]), ExpectationValueResult(value=(0.243896484375+0j), variance=(11.714251935482025+0j), confidence_interval=[((0.18392719721659406+0j), (0.30386577153340594+0j))]), ExpectationValueResult(value=(0.841552734375+0j), variance=(12.361730754375458+0j), confidence_interval=[((0.7801707906113339+0j), (0.9029346781386661+0j))])])

### Mitigator

In [22]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(backend, mitigation="tensored", refresh=1, shots=2000, mit_pattern=[[0], [1]])
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")
print(f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")

w/o mitigation shots=2000, result=ExpectationValueResult(value=(1.5380000000000003+0j), variance=(12.102556+0j), confidence_interval=[((1.41597261148584+0j), (1.6600273885141605+0j))])
w/  mitigation shots=2000, result=ExpectationValueResult(value=(1.8415135056830598+0j), variance=(11.461711322019683+0j), confidence_interval=[((1.7242585449985073+0j), (1.9587684663676124+0j))])
